In [20]:
import pandas as pd
import numpy as np

from datetime import timedelta
from datetime import datetime


In [21]:
data = pd.read_csv('data/2023.csv')

In [15]:
data = pd.read_csv('data/2022.csv')
data = data[data['Actividad_Realizada'] == 'Muestreo']
data['Etapa_Fasica'].value_counts()


Etapa_Fasica
Solitaria               840
Trasciens Congregans     43
Name: count, dtype: int64

In [16]:

data['Recomendaciones'].value_counts()


Recomendaciones
PEP6                                            49
PEP 06                                          16
SE APLICO CONTROL BIOLOGICO AEREO EN 50 HAS.     1
SE APLICO CONTROL BIOLOGICO AEREO EN 77 HAS      1
evaluacion de aplicacion con drone               1
área quemada                                     1
Àrea quemada                                     1
àrea con aplicaciòn quìmica                      1
evalucion de aplicacion con drone                1
Name: count, dtype: int64

In [22]:
data = pd.read_csv('data/2023.csv')
data = data[data['Actividad_Realizada'] == 'Exploración']
data['Resultado'].value_counts()

Resultado
Negativo      6631
Positivo      1168
En proceso      16
Name: count, dtype: int64

In [23]:
data.drop(data[data['Resultado']=='En proceso'].index, inplace=True)
data['Resultado'].value_counts()

Resultado
Negativo    6631
Positivo    1168
Name: count, dtype: int64

In [27]:
data_output = data[['Fecha','Latitud','Longitud','Resultado']]
data_output.loc[:,'Fecha'] = data_output['Fecha'].astype('datetime64[s]')

data_output['Fecha_inicio'] = data_output['Fecha'] - timedelta(days=96)
data_output

C:\Users\seder\AppData\Local\Temp\ipykernel_23280\1378835331.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_output['Fecha_inicio'] = data_output['Fecha'] - timedelta(days=96)


,Fecha,Latitud,Longitud,Resultado,Fecha_inicio
0,2023-01-03 00:00:00,20.85595,-90.25643,Negativo,2022-09-29 00:00:00
1,2023-01-03 00:00:00,20.85498,-90.26168,Negativo,2022-09-29 00:00:00
2,2023-01-03 00:00:00,20.81349,-90.19725,Negativo,2022-09-29 00:00:00
3,2023-01-03 00:00:00,20.80072,-90.19945,Negativo,2022-09-29 00:00:00
4,2023-01-03 00:00:00,20.83347,-90.21466,Negativo,2022-09-29 00:00:00
...,...,...,...,...,...
7974,2023-12-28 00:00:00,20.97265,-88.97319,Negativo,2023-09-23 00:00:00
7975,2023-12-29 00:00:00,20.98016,-88.96617,Negativo,2023-09-24 00:00:00
7976,2023-12-29 00:00:00,20.95807,-88.89866,Negativo,2023-09-24 00:00:00
7977,2023-12-28 00:00:00,20.99703,-88.97471,Negativo,2023-09-23 00:00:00


In [33]:
#idx = pd.date_range('2021-09-20', '2021-09-20')

def get_series(folder,field_name,row,names,xmin,xmax):
    df = pd.read_csv(folder +field_name+'/' +  str(row['Latitud']) + '_' + str(row['Longitud']) + '.csv')
    
    df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d')  
    df = df.set_index('date').asfreq('D', method='ffill').reset_index()
    
    mask = (df['date'] > row['Fecha_inicio']) & (df['date'] <= row['Fecha'])
    df = df.loc[mask]
    df[names] = (df[names] - xmin )/(xmax-xmin)
    list_f = df[names].values.tolist()
    len_list = len(list_f)  
    if len_list == 0:
        list_f = [0]*96
    if len_list != 96:
        #print(len(list_f))
        for i in range(len_list,96):
            list_f.append(list_f[len_list-1])
        #print(list_f)
        
        #print(len(list_f))
        #print(row)

    return list_f



def get_series_group(folder,field_name,row,names,xmin,xmax):
    df = pd.read_csv(folder +field_name+'/' +  str(row['Latitud']) + '_' + str(row['Longitud']) + '.csv')
    df['date'] = pd.to_datetime(df['date'],format='%Y%m%d')  
    #df = df.set_index('date').asfreq('D', method='ffill').reset_index()
    mask = (df['date'] > row['Fecha_inicio']) & (df['date'] <= row['Fecha'])
    df = df.loc[mask]
    list_f = df[names].values.tolist()  
    len_list = len(list_f)  
    if len_list == 0:
        list_f = [0]*96
        len_list = len(list_f)  
    if len_list != 96:
        print('len list')
        print(len(list_f))
        for i in range(len_list,96):
            list_f.append(list_f[len_list-1])

        print('len list f')
        print(len(list_f))
        #print(row)

    return list_f

#get_series(data_output.iloc[0])x['NDVI'].tolist()




In [34]:
print('NDVI')
data_output['NDVI'] = data_output.apply(lambda row: get_series('gee-coordinates-2023/','NDVI',row,'NDVI',0,1), axis=1)

print('LST_Day_1km')
data_output['LST_Day_1km'] = data_output.apply(lambda row: get_series('gee-coordinates-2023/','LST_Day_1km',row,'LST_Day_1km',0,1), axis=1)

print(['Psurf_f_inst','Qair_f_inst','Rainf_tavg','Tair_f_inst','Wind_f_inst','RootMoist_inst'])
features2 =  ['Psurf_f_inst','Qair_f_inst','Rainf_tavg','Tair_f_inst','Wind_f_inst','RootMoist_inst']
for feature in features2:
    data_output[feature] = data_output.apply(lambda row: get_series_group('gee-coordinates-2023/','SOIL',row,feature,0,1), axis=1)

print(['prcp','srad','tmax','tmin','vp'])
features =  ['prcp','srad','tmax','tmin','vp']
for feature in features:
    data_output[feature] = data_output.apply(lambda row: get_series_group('gee-coordinates-2023/','DAYMET_V4',row,feature,0,1), axis=1)

dataset = data_output[features+features2+['LST_Day_1km','NDVI','Resultado']]
dataset.to_csv('data/2023-dataset.csv', index=False) 
#dataset.to_csv('data/2022-dataset.csv', index=False) 




NDVI


C:\Users\seder\AppData\Local\Temp\ipykernel_23280\3532393074.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_output['NDVI'] = data_output.apply(lambda row: get_series('gee-coordinates-2023/','NDVI',row,'NDVI',0,1), axis=1)


LST_Day_1km


C:\Users\seder\AppData\Local\Temp\ipykernel_23280\3532393074.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_output['LST_Day_1km'] = data_output.apply(lambda row: get_series('gee-coordinates-2023/','LST_Day_1km',row,'LST_Day_1km',0,1), axis=1)


['Psurf_f_inst', 'Qair_f_inst', 'Rainf_tavg', 'Tair_f_inst', 'Wind_f_inst', 'RootMoist_inst']


C:\Users\seder\AppData\Local\Temp\ipykernel_23280\3532393074.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_output[feature] = data_output.apply(lambda row: get_series_group('gee-coordinates-2023/','SOIL',row,feature,0,1), axis=1)
C:\Users\seder\AppData\Local\Temp\ipykernel_23280\3532393074.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_output[feature] = data_output.apply(lambda row: get_series_group('gee-coordinates-2023/','SOIL',row,feature,0,1), axis=1)
C:\Users\seder\AppData\Local\T

['prcp', 'srad', 'tmax', 'tmin', 'vp']


C:\Users\seder\AppData\Local\Temp\ipykernel_23280\3532393074.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_output[feature] = data_output.apply(lambda row: get_series_group('gee-coordinates-2023/','DAYMET_V4',row,feature,0,1), axis=1)
C:\Users\seder\AppData\Local\Temp\ipykernel_23280\3532393074.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_output[feature] = data_output.apply(lambda row: get_series_group('gee-coordinates-2023/','DAYMET_V4',row,feature,0,1), axis=1)
C:\Users\seder\AppDa

: 

In [5]:

dataset = pd.read_csv('data/2023-dataset.csv')

features_all = ['LST_Day_1km','NDVI']+['Psurf_f_inst','Qair_f_inst','Rainf_tavg','Tair_f_inst','Wind_f_inst','RootMoist_inst']+['prcp','srad','tmax','tmin','vp']



#dataset = data_output[['Resultado']+features+features2]

def convert(row,feature,st,end):
    list_parsed = eval(row[feature])[st:end]
    if np.sum(list_parsed) == 0:
        return 0.0
    else:
        return np.mean(list_parsed)

feature_dataset = pd.DataFrame()
feature_dataset['Resultado'] = dataset['Resultado']
for feature in features_all:
    print(feature)
    feature_dataset[feature + '_1'] = dataset.apply(lambda row: convert(row,feature,0,12),axis=1)
    feature_dataset[feature + '_2'] = dataset.apply(lambda row: convert(row,feature,12,24),axis=1)
    feature_dataset[feature + '_3'] = dataset.apply(lambda row: convert(row,feature,24,36),axis=1)
    feature_dataset[feature + '_4'] = dataset.apply(lambda row: convert(row,feature,36,48),axis=1)
    feature_dataset[feature + '_5'] = dataset.apply(lambda row: convert(row,feature,48,60),axis=1)
    feature_dataset[feature + '_6'] = dataset.apply(lambda row: convert(row,feature,60,72),axis=1)
    feature_dataset[feature + '_7'] = dataset.apply(lambda row: convert(row,feature,72,84),axis=1)
    feature_dataset[feature + '_8'] = dataset.apply(lambda row: convert(row,feature,84,96),axis=1)

feature_dataset.to_csv('data/2023-feature-dataset.csv', index=False) 
feature_dataset.describe()

LST_Day_1km
NDVI
Psurf_f_inst
Qair_f_inst
Rainf_tavg
Tair_f_inst
Wind_f_inst
RootMoist_inst
prcp
srad
tmax
tmin
vp


/var/folders/9n/kxcs2s054c32h_5hwhhg4bd40000gn/T/ipykernel_45971/3348667295.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feature_dataset[feature + '_4'] = dataset.apply(lambda row: convert(row,feature,36,48),axis=1)
/var/folders/9n/kxcs2s054c32h_5hwhhg4bd40000gn/T/ipykernel_45971/3348667295.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feature_dataset[feature + '_5'] = dataset.apply(lambda row: convert(row,feature,48,60),axis=1)
/var/folders/9n/kxcs2s054c32h_5hwhhg4bd40000gn/T/ipykernel_45971/3348667295.py:25: Per

,LST_Day_1km_1,LST_Day_1km_2,LST_Day_1km_3,LST_Day_1km_4,LST_Day_1km_5,LST_Day_1km_6,LST_Day_1km_7,LST_Day_1km_8,NDVI_1,NDVI_2,...,tmin_7,tmin_8,vp_1,vp_2,vp_3,vp_4,vp_5,vp_6,vp_7,vp_8
count,7799.000000,7799.000000,7799.000000,7799.000000,7799.000000,7799.000000,7799.000000,7799.000000,7799.000000,7799.000000,...,7799.000000,7799.000000,7799.000000,7799.000000,7799.000000,7799.000000,7799.000000,7799.000000,7799.000000,7799.000000
mean,15255.213671,15254.206800,15250.388875,15253.502297,15258.535752,15258.239603,15253.777974,15252.833216,6613.497436,6634.931102,...,22.279376,22.319076,2097.534073,2142.753447,2189.859193,2224.726563,2242.686664,2239.486492,2206.773433,2173.042916
std,210.915704,214.654367,216.510660,214.858517,207.683393,192.590453,179.731659,174.326298,1421.508590,1415.829049,...,2.266660,2.251899,655.833698,629.232122,585.235645,551.743151,540.376932,594.799377,666.555953,726.643837
min,14800.333333,14747.916667,14699.916667,14827.500000,14813.083333,14786.583333,14803.916667,14664.250000,1977.666667,1294.000000,...,16.757500,16.783333,896.429164,894.070002,901.710002,956.823334,906.647502,888.688339,882.500839,897.375824
25%,15056.000000,15055.166667,15052.333333,15057.916667,15079.875000,15107.583333,15117.833333,15129.500000,5480.333333,5493.041667,...,20.137083,20.363333,1787.406657,1909.143326,1937.985835,1932.068746,1934.877914,1933.096252,1895.725408,1698.148336
50%,15239.500000,15240.916667,15222.666667,15222.916667,15221.000000,15229.833333,15231.166667,15241.000000,6841.666667,6946.333333,...,23.197500,23.208333,2190.267487,2223.169993,2220.601685,2214.755809,2232.556712,2253.450043,2281.454976,2325.236674
75%,15424.500000,15430.875000,15434.166667,15434.250000,15428.500000,15393.875000,15378.208333,15366.791667,7800.000000,7787.333333,...,24.255000,24.365417,2585.487096,2600.089152,2609.442922,2623.179159,2636.783335,2686.196248,2727.860819,2849.303772
max,15889.333333,15866.583333,15876.333333,15843.500000,15870.750000,15851.083333,15834.250000,15816.500000,9799.833333,9717.000000,...,25.385000,25.409167,3228.960815,3178.584167,3191.529175,3203.571696,3193.013306,3194.997498,3206.190002,3208.705831
